In [912]:
import pandas as pd
df = pd.read_csv('Salary Dataset.csv')
pd.set_option('display.max_rows', 1100)
#df.head()

In [913]:
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' or '/hr' in x.lower() else 0)
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)
df['monthly'] = df['Salary Estimate'].apply(lambda x: 1 if '/mo' in x.lower() else 0)

In [914]:
df = df[df['Salary Estimate'] != '-1']
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
minus_kd = salary.apply(lambda x: x.replace('K', '').replace('$', ''))

In [915]:
# Deletes 'per hour', 'employer provided salary' and '/yr'
clean_salary = minus_kd.apply(lambda x: x.lower().replace('per hour', '').replace('employer provided salary', '').replace('Employer est.', '').replace('/mo', '').replace('/yr', '').strip())

In [916]:
# df.employer_provided.value_counts()
df =df.drop('employer_provided', axis=1)

In [917]:
# df.hourly.value_counts()
df = df.drop('hourly', axis=1)

In [918]:
df['min_salary'] = clean_salary.apply(lambda x: x.split('–')[0].strip() if pd.notna(x) else '-1')
df['max_salary'] = clean_salary.apply(lambda x: x.split('–')[1].strip() if '–' in x and len(x.split('–')) > 1 else x.split('–')[0].strip())
df['min_salary'] = pd.to_numeric(df['min_salary'], errors='coerce').fillna(-1).astype(int)
df['max_salary'] = pd.to_numeric(df['max_salary'], errors='coerce').fillna(-1).astype(int)

In [919]:
# calculates the average salary
df['avg_salary'] = (df.min_salary + df.max_salary)/2

In [920]:
df['min_salary'] = df.apply(lambda x: x.min_salary*12 if x.monthly == 1 else x.min_salary, axis = 1)
df['max_salary'] = df.apply(lambda x: x.max_salary*12 if x.monthly == 1 else x.max_salary, axis = 1)
#df[df.monthly == 1][['min_salary', 'max_salary']]

In [921]:
df =df.drop('monthly', axis=1)

In [922]:
# Erases the '\n' in the job description.
df['Job Description'] = df['Job Description'].astype(str).apply(lambda x: x.replace('\n', ' ').replace('nan', '-1'))
# Job description length
df['desc_len'] = df['Job Description'].apply(lambda x: len(x) if len(x) != 2 else '-1')

In [923]:
def split_location(location):
    if 'remote' in location.lower():
        return '-1', '-1', 1, '-1' 
    else:
        parts = location.split(',') 
        if len(parts) == 2:
            city, state = parts
            country = '-1' 
        elif len(parts) == 1:  
            city = state = '-1' 
            country = parts[0]  
        else:  
            city = state = country = '-1'
        return city.strip(), state.strip(), 0, country.strip()  # Gibt die Ergebnisse ohne Leerzeichen zurück und 'Remote' zu 0, wenn nicht remote

new_columns = df['Location'].apply(lambda x: pd.Series(split_location(x), index=['City', 'State', 'Remote', 'Country']))

df = pd.concat([df, new_columns], axis=1)

In [924]:
df = df[(df['Country'] == '-1') | (df['Country'] == 'United States')]
# df['Country'].value_counts()

In [925]:
# df.Remote.value_counts()

In [926]:
# df.City.value_counts()

In [927]:
# df.State.value_counts()

In [928]:
# df.Country.value_counts()

In [929]:
def split_revenue(revenue):
    if 'unknown' in revenue.lower() or 'non-applicable' in revenue.lower() or not any(char.isdigit() for char in revenue):
        return -1, -1, -1 
    
    revenue = revenue.lower().replace('$', '').replace('(usd)', '').strip()

    parts = revenue.split('to')
    if len(parts) == 1:
        min_part = max_part = parts[0]
        if 'billion' in min_part:
            min_revenue = max_revenue = convert_to_int(min_part.replace('billion', '').replace('+', '').strip()) * 1000
        else:
            min_revenue = max_revenue = convert_to_int(min_part.replace('million', '').strip())
        
    elif len(parts) == 2:
        min_part, max_part = parts

        if 'billion' in max_part:
            min_revenue = convert_to_int(min_part.replace('million', '').strip()) if 'million' in min_part else convert_to_int(min_part.strip()) * 1000
            max_revenue = convert_to_int(max_part.replace('billion', '').strip()) * 1000
        
        else:  
            min_revenue = convert_to_int(min_part.replace('million', '').strip())
            max_revenue = convert_to_int(max_part.replace('million', '').strip())
            
    else:
        min_revenue = max_revenue = avg_revenue = -1
    
    avg_revenue = (min_revenue + max_revenue) // 2  
    
    return min_revenue, max_revenue, avg_revenue

def convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return -1 

df[['Min Revenue', 'Max Revenue', 'Avg Revenue']] = df['Revenue'].apply(lambda x: pd.Series(split_revenue(x)))

In [930]:
revenue_df = df[df['Revenue'] != '-1']


In [931]:
df['Founded'] = pd.to_numeric(df['Founded'], errors='coerce')

df['Founded'] = df['Founded'].fillna(-1).astype(int)

def calculate_age(founded):
    if founded > 0 and founded <= 2024:
        return 2024 - founded 
    else:
        return -1

df['Age'] = df['Founded'].apply(calculate_age)

In [932]:
df['python_yn'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
# df.python_yn.value_counts()

In [933]:
df['R_yn'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
# df.R_yn.value_counts()

In [934]:
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
# df.spark.value_counts()

In [935]:
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
# df.spark.value_counts()

In [936]:
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
# df.excel.value_counts()

In [937]:
# df.columns

In [938]:
df = df.drop('Unnamed: 0', axis=1)

In [939]:
df.to_csv('salary_data_cleaned')